In [ ]:
from flask import Flask, request, jsonify, render_template, session, url_for, redirect
import numpy as np
from flask_wtf import FlaskForm
from wtforms import TextField, SubmitField
import tensorflow
from keras.models import load_model
import joblib


from tensorflow.keras.models import load_model
import joblib

In [ ]:
def return_prediction(model, scaler, sample_json):

    T = sample_json["T"]
    TM = sample_json["TM"]
    Tm = sample_json["Tm"]
    SLP = sample_json["SLP"]
    H = sample_json["H"]
    VV = sample_json["VV"]
    V = sample_json["V"]
    VM = sample_json["VM"]

    air = [[T,TM,Tm,SLP,H,VV,V,Vm]]
    air = scaler.transform(air)

    
    return model.predict(air)[0]

In [ ]:
app = Flask(__name__)
app.config['AIR_QUALITY'] = 'airquality'

In [ ]:
class AirForm(FlaskForm):
    t = TextField("AVG. ANNUAL TEMPERATURE")
    tM = TextField("ANNUAL AVERAGE MAXIMUM TEMPERATURE")
    tm = TextField("AVG. ANNUAL MINIMUM TEMPERATURE")
    slp = TextField("AIR PRESSURE")
    h = TextField("AVERAGE RELATIVE HUMIDITY")
    vv = TextField("AVERAGE VISIBILITY")
    v = TextField("ANNUAL AVERAGE WIND SPEED")
    vm = TextField("MERCURY VAPOUR")

    submit = SubmitField("Analyse")

In [ ]:
@app.route("/", methods=['GET', 'POST'])
def index():
    form = AirForm()
    if form.validate_on_submit():
        session['t'] = form.t.data
        session['tM'] = form.tM.data
        session['tm'] = form.tm.data
        session['slp'] = form.slp.data
        session['h'] = form.h.data
        session['vv'] = form.vv.data
        session['v'] = form.v.data
        session['vm'] = form.vm.data

        return redirect(url_for("prediction"))
    return render_template('home.html', form=form)

In [ ]:

model = load_model('ANN1.h5')
scaler = joblib.load("model1.pkl",'r')

In [ ]:
@app.route('/prediction')
def prediction():
    content = {}

    content['AVG. ANNUAL TEMPERATURE'] = float(session['t'])
    content['ANNUAL AVERAGE MAXIMUM TEMPERATURE'] = float(session['tM'])
    content['AVG. ANNUAL MINIMUM TEMPERATURE'] = float(session['tm'])
    content['AIR PRESSURE'] = float(session['slp'])
    content['AVERAGE RELATIVE HUMIDITY'] = float(session['h'])
    content['AVERAGE VISIBILITY'] = float(session['vv'])
    content['ANNUAL AVERAGE WIND SPEED'] = float(session['v'])
    content['MERCURY VAPOUR'] = float(session['vm'])

    results = return_prediction(model,scaler, content)

    return render_template('prediction.html', results=results)

In [ ]:
if __name__ == '__main__':
    app.run(debug=True)